In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
#Normal imports
import os
import tensorflow as tf
import keras
import matplotlib.pyplot as plt

#For imagedatagen
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#For the custom model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

#For the test train split
from sklearn.model_selection import train_test_split

#For the optimizer
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa

In [ ]:
data = 'C:/Users/Student/sreekar/augemted_dataset'

## Image datagen

In [ ]:
# Creating a Data Generator Object
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.2,
    interpolation_order=1,
    dtype=None
)

In [ ]:
train_set = datagen.flow_from_directory(
    data,
    target_size = (256, 256),
    batch_size = 64,
    subset = 'training',
    class_mode='categorical',
    shuffle= False
)

In [ ]:
valid_set = datagen.flow_from_directory(
    data,
    target_size = (256, 256),
    batch_size = 64,
    subset = 'validation',
    class_mode='categorical',
    shuffle= False
)

In [ ]:
# import os
# from IPython.display import display, Image

# def display_images_from_class(dataset_path, class_name):
#     class_path = os.path.join(dataset_path, class_name)

#     try:
#         # Get a list of all image files in the class directory
#         image_files = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

#         if not image_files:
#             print(f"No images found in the {class_name} class.")
#             return

#         # Display each image
#         for image_file in image_files:
#             image_path = os.path.join(class_path, image_file)
#             display(Image(filename=image_path, format='png'))

#     except FileNotFoundError:
#         print(f"Error: Class {class_name} not found in the dataset.")
#     except Exception as e:
#         print(f"Error: An unexpected error occurred - {e}")

# # Example usage:
# dataset_path = 'C:/Users/Student/sreekar/augemted_dataset'  # Replace with the actual path to your dataset
# class_name_to_display = 'C:/Users/Student/sreekar/augemted_dataset/A1'  # Replace with the class name you want to display
# display_images_from_class(dataset_path, class_name_to_display)


## model

In [ ]:
# data_augmentation = keras.Sequential(
#   [
#     layers.RandomFlip("horizontal",
#                       input_shape=(224,224,3)),
#     layers.RandomRotation(0.1),
#     layers.RandomZoom(0.1),
#   ]
# )

In [ ]:
model = Sequential()

# Input Layers
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Dense Layers
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

# Output Layer
model.add(Dense(units = 673, activation = 'softmax'))

In [ ]:
model.compile(optimizer=Adam(lr = 0.00001),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), tfa.metrics.F1Score(num_classes=673, average='macro')])

model.summary()

In [ ]:
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)

In [ ]:
# Train the model
hist = model.fit(train_set, validation_data=valid_set,
            epochs = 20, steps_per_epoch=len(train_set),
                    validation_steps=len(valid_set), callbacks = [tensorboard_callback])